# Moteur DC

Soit le moteur DC de fonction de transfert:

$$ H(s) = \frac{0.92}{0.3s + 1} \frac{1.658}{s} $$

On demande ce qui suit:

1. Déterminer le modèle d'état échantillonné pour $h=10\, \mathrm{ms}$ avec $y$ la position angulaire du moteur
2. Vérifier la stabilité du système échantillonné
3. Vérifier si le système est commandable et transformer le modèle sous la forme canonique commandable
4. Vérifier si le système est observable et transformer le modèle sous la forme canonique observable

---

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('../my_params.mplstyle')

import numpy as np
import scipy.integrate as spint
import control

1. Déterminer le modèle d'état échantillonné pour $h=10\, \mathrm{ms}$

L'équation différentielle est obtenue à partir de la fonction de transfert:

\begin{align}
  \Theta(s) \left(0.3 s^2 + s \right) &= 1.52536\; U(s) \\
  0.3 \frac{d^2\theta(t)}{dt^2} + \frac{d\theta(t)}{dt} &= 1.52536\; u(t) \\
  \frac{d^2\theta(t)}{dt^2} &= - 3.33333\; \frac{d\theta(t)}{dt} + 5.0845\; u(t)
\end{align}

Le modèle d'état analogique obtenu est alors:

$$
  \left\{ \begin{array}{l}
    \left[ \begin{array}{c}
      \dot{x}_1 \\
      \dot{x}_2 
    \end{array} \right] = \left[ \begin{array}{cc}
      0 & 1 \\
      0 & -3.33333 
    \end{array} \right] \left[ \begin{array}{c}
      x_1 \\
      x_2
    \end{array} \right] + \left[ \begin{array}{c}
      0 \\
      5.0845
    \end{array} \right] u \\
    y = \left[ \begin{array}{cc}
      1 & 0
    \end{array} \right] \left[ \begin{array}{c}
      x_1 \\
      x_2
    \end{array} \right] + 0\; u
  \end{array} \right.
$$

A partir de cette expression, le reste peut être effectué numériquement:

In [2]:
A = np.array([[0, 1], [0, -3.33333]])
B = np.array([[0], [5.0845]])
C = np.array([1, 0])
D = 0

h = 0.01

motor_ss = control.ss(A, B, C, D)
motor_ss_d = control.c2d(motor_ss, h)

print(motor_ss_d)

A = [[1.         0.00983517]
 [0.         0.96721613]]

B = [[0.00025142]
 [0.05000692]]

C = [[1. 0.]]

D = [[0.]]

dt = 0.01



#### Remarque:

Il est possible en général de faire calculer un modèle d'état directement à partir d'une fonction de transfert:

In [3]:
print(control.tf2ss(1.52536, [0.3, 1, 0]))

A = [[-3.33333333 -0.        ]
 [ 1.          0.        ]]

B = [[1.]
 [0.]]

C = [[0.         5.08453333]]

D = [[0.]]



On retrouve une description équivalente à celle déterminée à la main.

---

2. Vérifier la stabilité du système échantillonné

Il suffit pour cela de calculer et analyser les valeurs propres de la matrice $\mathbf{F}$:

In [4]:
print(np.linalg.eig(motor_ss_d.A)[0])

[1.         0.96721613]


Les 2 pôles étant réels positifs, il n'y a pas de mode sonnette ni d'oscillations. Par contre, le système possède bien une intégration et un pôle asymptotiquement stable. Un état est donc stable et l'autre à la limite de stabilité.

---

3. Vérifier si le système est commandable et transformer le modèle sous la forme canonique commandable

Calculons la matrice de gouvernabilité:

In [5]:
Com = control.ctrb(motor_ss_d.A, motor_ss_d.B)
print('Le système est gouvernable' if np.linalg.matrix_rank(Com) == 2 else 'Le système n\'est pas gouvernable')

Le système est gouvernable


In [6]:
motor_ss_d_g = control.canonical_form(motor_ss_d, 'reachable')
print(motor_ss_d_g)

(A = [[ 1.96721613 -0.96721613]
 [ 1.          0.        ]]

B = [[1.]
 [0.]]

C = [[0.00025142 0.00024865]]

D = [[0.]]

dt = 0.01
, matrix([[1999.72315998,    9.94306902],
        [1999.72315998,  -10.05416258]]))


---

4. Vérifier si le système est observable et transformer le modèle sous la forme canonique observable

Calculons la matrice d'observabilité:

In [7]:
Obs = control.obsv(motor_ss_d.A, motor_ss_d.C)
print('Le système est observable' if np.linalg.matrix_rank(Obs) == 2 else 'Le système n\'est pas observable')

Le système est observable


In [8]:
motor_ss_d_g = control.canonical_form(motor_ss_d, 'observable')
print(motor_ss_d_g)

(A = [[ 1.96721613  1.        ]
 [-0.96721613  0.        ]]

B = [[0.00025142]
 [0.00024865]]

C = [[1. 0.]]

D = [[0.]]

dt = 0.01
, matrix([[ 1.        ,  0.        ],
        [-0.96721613,  0.00983517]]))
